In [1]:
import pandas as pd
from pathlib import Path
import os, sys
import plotly.graph_objects as go
import json

# PROJECT_ROOT = Path(os.path.abspath('')).parents[1]
# sys.path.insert(0,PROJECT_ROOT)

module_path = os.path.abspath('..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)


from utils import LINE_AND_MARKER, create_layout, create_trace, comparison_plot, normalise_series_to_first_value

# Input Data

In [2]:
DATA_DIR = Path(os.path.abspath('')).parents[1] / "data"
FARE_PRICES_DIR = DATA_DIR / "fares"
STOCK_PRICES_DIR = DATA_DIR / "stocks"
STOCK_PRICES_DIR

PosixPath('/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks')

In [3]:
tflFares = pd.read_csv(FARE_PRICES_DIR / "raw" / "TfLHistoricalFares2000to2025.csv")
tflFares

,year,Single_Z1_to_Z4_Cash,Single_Z1_to_Z4_Oyster_Peak,Bus_Cash,Single_Bus_Oyster,Cap_Bus_Tram,Travelcard_Z1_to_Z4,Cap_Z1_to_Z4_PAYG,Travelcard_7_Day_Z1_to_Z4,Weekly_Bus_And_Tram_Pass
0,2000,2.6,NaN,1.0,NaN,NaN,NaN,NaN,26.8,11.5
1,2001,2.7,NaN,1.0,NaN,NaN,NaN,NaN,27.6,9.5
2,2002,2.7,NaN,1.0,NaN,NaN,6.8,NaN,28.1,8.5
3,2003,2.8,NaN,1.0,NaN,NaN,7.0,NaN,28.4,8.5
4,2004,3.0,2.8,1.0,0.70,NaN,7.3,NaN,29.2,9.5
5,2005,2.8,2.5,1.2,1.00,3.00,8.0,NaN,30.4,11.0
6,2006,3.0,2.5,1.5,1.00,3.00,8.4,NaN,31.6,13.5
7,2007,4.0,2.5,2.0,1.00,3.00,9.0,NaN,33.2,14.0
8,2008,4.0,2.5,2.0,0.90,3.00,9.4,NaN,34.6,13.0
9,2009,4.0,2.8,2.0,1.00,3.30,10.0,NaN,36.8,13.8


# Charts

In [4]:
def normalise_single_series_to_100(df):
    return (df - df.min())/(df.max()-df.min()) * 100

## Tube Fares

In [5]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_series_to_first_value(tflFares["Single_Z1_to_Z4_Oyster_Peak"][10:]) * 100 - 100, label="Contactless ticket"))
fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_series_to_first_value(tflFares["Single_Z1_to_Z4_Cash"][10:]) * 100 - 100, label="Paper ticket"))
# fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_single_series_to_100(tflFares["singleZ1to4OysterPeak"][10:]), label="Contactless ticket"))
# fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_single_series_to_100(tflFares["singleZ1to4Cash"][10:]), label="Paper ticket"))
# LAYOUT
layout = create_layout(title="Normalised TfL Tube Fares <br><sup>Zone 1-4 travel during peak hours</sup>")
fig.update_layout(layout)

fig.show()

In [6]:
# Get pay-as-you-go caps as well
# How many journeys are required to reach the cap? -> could put it as barchart underneath
# tflFares['capZ1to4PAYG']/tflFares['singleZ1to4OysterPeak']

# fig = go.Figure()
# # TRACES
# fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['capZ1to4PAYG'], label="PAYG Cap"))
# # fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['travelcard7DayZ1to4'], label="travelcard7DayZ1to4"))
# # LAYOUT
# layout = create_layout(title="TfL Tube Fares <br><sup>Zone 1-4 travel during peak hours</sup>")
# fig.update_layout(layout)

# fig.update_xaxes(range=[2012,2025])

# fig.show()

## FTSE100

In [7]:
data = json.load(open(STOCK_PRICES_DIR / "ftse100from2000to2025.json"))
ftse100 = pd.DataFrame(data['data'])
ftse100
# Month end value are reported from 2000 Jan to 2025 March + mid April

,_DATE_END,LOW_1,CLOSE_PRC,HIGH_1,OPEN_PRC
0,2000-01-31,6246.8,6268.54,6930.2,6930.2
1,2000-02-29,5972.7,6232.56,6450.9,6268.5
2,2000-03-31,6232.6,6540.22,6770.4,6232.6
3,2000-04-30,5915.2,6327.43,6586.3,6540.2
4,2000-05-31,5991.9,6359.35,6419.9,6327.4
...,...,...,...,...,...
299,2024-12-31,8002.34,8173.02,8388.37,8287.3
300,2025-01-31,8160.6,8673.96,8692.84,8173.02
301,2025-02-28,8520.2,8809.74,8820.93,8673.96
302,2025-03-31,8481.11,8582.81,8908.82,8809.74


In [8]:
ftse100['_DATE_END'] = pd.to_datetime(ftse100['_DATE_END'])
ftse100['year'] = ftse100['_DATE_END'].dt.year
# ftse100['year'] = ftse100['_DATE_END'].apply(lambda x: int(x.split('-')[0]))

ftse_yearly_max_prices = ftse100.groupby("year", as_index=False).max() # HALO this is stupid
ftse_yearly_max_prices = ftse_yearly_max_prices.sort_values("year")

ftse_yearly_min_prices = ftse100.groupby("year", as_index=False).min()
ftse_yearly_min_prices = ftse_yearly_min_prices.sort_values("year")

ftse_yearly_min_prices.head()

,year,_DATE_END,LOW_1,CLOSE_PRC,HIGH_1,OPEN_PRC
0,2000,2000-01-31,5915.2,6142.19,6419.9,6142.2
1,2001,2001-01-31,4219.8,4903.39,5279.8,4903.4
2,2002,2002-01-31,3609.9,3721.75,4197.5,3721.8
3,2003,2003-01-31,3277.5,3567.41,3747.0,3567.4
4,2004,2004-01-31,4283.0,4385.67,4487.9,4385.7


## Combined Charts

In [9]:
# Create step non-decreasing step function
fig = go.Figure()

fares_normalised = normalise_series_to_first_value(tflFares["Single_Z1_to_Z4_Oyster_Peak"][10:]) * 200 - 200
stocks_normalised = normalise_series_to_first_value(ftse_yearly_max_prices['HIGH_1'].astype(float)[10:]) * 200 - 200
# TRACES
fig.add_trace(create_trace(
    x=tflFares['year'][10:],
    y=fares_normalised, 
    label="Contactless ticket (PAYG)",
    line_shape="hv"
))
fig.add_trace(create_trace(
    x=ftse_yearly_max_prices['year'][10:], 
    y=stocks_normalised, 
    label="FTSE100 annual highs",
    # line_shape="hv"
))

# LAYOUT
layout = create_layout(title="TfL Tube Fares vs the FTSE100<br><sup>Prices are normalised to the same starting point of 1</sup>")
fig.update_layout(layout)

fig.show()

In [10]:
((fares_normalised - stocks_normalised) > 0).sum()
# 11 out of 15 years the tube prices rose above the FTSE100

# Reselling your tube tickets would reap in more revenue than investing it in the UK economy
#  through a stock ISA. Sadiq, Rachel - wanna chip in?

np.int64(11)

### Plot for Streamlit Dashboard

In [11]:
combined_fares = pd.read_csv(FARE_PRICES_DIR / "processed" / "combined_transport_fares.csv")
combined_stocks = pd.read_csv(STOCK_PRICES_DIR / "combined_indices.csv")
# Time to plot dynamically. Variables
# - Transport modes selected
# - Indices selected
# - Year range
# How to distinguish between transport and stocks? Make transport costs a step function

In [12]:
indices_keys = combined_stocks.columns.to_list()
indices_keys.remove("year")
print(indices_keys)

transportation_keys = combined_fares.columns.to_list()
transportation_keys.remove("year")
print(transportation_keys)

['HangSeng', 'FTSE100', 'Nikkei225', 'SnP500', 'DAX']
['Bus_and_Coach', 'Rail', 'Single_Z1_to_Z4_Cash', 'Single_Z1_to_Z4_Oyster_Peak', 'Bus_Cash', 'Single_Bus_Oyster', 'Cap_Bus_Tram', 'Travelcard_Z1_to_Z4', 'Cap_Z1_to_Z4_PAYG', 'Travelcard_7_Day_Z1_to_Z4', 'Weekly_Bus_And_Tram_Pass']


In [13]:
selected_stocks = indices_keys[:3]
selected_fares = transportation_keys[:3]
comparison_plot(combined_stocks, combined_fares, selected_fares, selected_stocks, 2010, 2020)

# Ideas, thoughts

In [14]:
# Create plotly/streamlit chart where one can select the following from a dropdown
# - Stock Index -> Get S&P 500
# Tranportation mode -> we have average rail and bus/coach fares too from 

# Also need a dropdown for selecting years (2000-2025) and deal with missing values pls
# Though fares go back to 1987, and so do most indeces -> Did a search in perplexity

# Sources
# - Bus & coach: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/docx/mm23
# - Train: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/docw/mm23
# - FTSE100: downloaded from WSJ Markets
# - S&P 500: downloaded from WSJ Markets
# - DAX: downloaded from WSJ Markets
# - Nikkei 225: downloaded from WSJ Markets
# - Hang Seng: downloaded from WSJ Markets

In [15]:
# Could factor in inflation using CPIH
# CPIH stands for the Consumer Prices Index including owner occupiers' housing costs, which is the UK's leading measure of inflation
# Source: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23
# cpih = pd.read_csv(DATA_DIR / "CPIH-annual-rate-1989-2024-by-year.csv", skiprows=7)
# n_years = 2024 - 1989 + 1
# cpih.iloc[:n_years].tail()

In [16]:
# Defo need wages + inflation tracker on the plot
# Average Weekly Earnings (AWE) comparison: 2000 - £305, 2012 - £457, 2025 - £711 -> 49.83%, 133.11%
# https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/timeseries/kab9/emp
# Median weekly earnings for full-time employees: 2012 - £506, 2025 - £728
# https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2024

# Code Cemetary

In [17]:
# fig = go.Figure()
# # TRACES
# fig.add_trace(create_trace(x=ftse_yearly_max_prices['year'][10:], y=ftse_yearly_max_prices['HIGH_1'].astype(float)[10:], label="Highest annual price"))
# fig.add_trace(create_trace(x=ftse_yearly_min_prices['year'][10:], y=ftse_yearly_min_prices['LOW_1'].astype(float)[10:], label="Lowest annual price"))
# # LAYOUT
# layout = create_layout(title="FTSE100<br><sup>UK's stock market's index innit</sup>", type="linear")
# fig.update_layout(layout)

# fig.show()

In [18]:
# fig = go.Figure()
# # TRACES
# fig.add_trace(create_trace(x=ftse_yearly_max_prices['year'][10:], y=normalise_single_series_to_100(ftse_yearly_max_prices['LOW_1'].astype(float)[10:]), label="Lowest annual price"))
# # LAYOUT
# layout = create_layout(title="Normalised FTSE100<br><sup>UK's stock market's index innit</sup>", type="linear")
# fig.update_layout(layout)
# # fig.update_xaxes(range=[2010,2025])
# fig.show()